# Inaugural project

## Purpose
In this project we'll seek to solve the Black-Scholes model of a simulated underlying asset $S$

## Theory
Consider a stock price given by:
$$S_t = S_0\cdot \exp{\left[\sum_{r=1}^Tr_i\right]}$$
where $S_0$ is given and:
$$r_i = \left(\mu-\frac{1}{2}\sigma^2\right)\Delta t+\sigma\sqrt{\Delta t}\epsilon, \quad \epsilon\sim N(0,1)$$
In discrete time we have $dt\approx\Delta t=1/252$ 

Given the the initial value $S_0$, the underlying stock $S_t$, and the risk free rate $r$ we can solve for the fair value of call- or put option with maturity date $T$ at time $t$. Note that $T$, $t$ is written as fractions of a trading year with 252 trading days.

We find the option prices:
$$c_t=S_tN(d_1)-K\cdot\text{e}^{-r(T-t)}N(d_2), \quad p_t=K\cdot\text{e}^{-r(T-t)}N(-d_2)-S_tN(-d_1)$$
where, $N(x)=\frac{1}{\sqrt{2\pi}}\int_{-\infty}^x\text{e}^{-x^2/2} \ dx$

In [1]:
## pip installments
# pip install math
# pip install plotly
# pip install scipy.stats

In [2]:
## Installments
import numpy as np
import matplotlib.pyplot as plt
from math import exp as exp
import plotly.express as px
import pandas as pd
from scipy.stats import norm

In [3]:
## Define a function that simulate the stock price
def stock_price_sim(S0=100, mu=0.01, sigma=0.08, T=252):
    """
    Simulation of stock price according to Black-Scholes model.
    
    The function takes following arguments:
    S0: Initial stock value
    mu: Average stock price return
    sigma: Stock volatility
    T: Length of stock price history
    """
    
    # Set seed
    np.random.seed(1)
    
    # Calculate dt≈Delta t=1/T
    dt = 1/T
    
    # Create list of standard normal distributed values
    epsilon = [np.random.normal(0,1) for x in range(T)]
    
    # Daily log returns
    r = [(mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * epsilon[i] for i in range(T)]
    
    # Stock price evolution
    S = []
    for t in range(T):
        S_t = S0 * exp(sum(r[0:t+1]))
        S.append(S_t)
        
    # Create dataframe from list
    S_df = pd.DataFrame(S, columns = ['Stock Price'])
        
    # Return stock price data
    return S_df   

In [16]:
## Plot stock
stock_price = stock_price_sim(T=252)
fig = px.line(stock_price, title='Evolution of the underlying stock price',
             labels=dict(value="Stock Price", index="$t$"))
fig.show()

In [5]:
## Define a option that will calculate price of eu call option on non dividend paying stock
def call_price(K, r, T, S_t):
    """
    This functions calculates the price on a european call option on a 
    non dividend paying stock and takes the arguments:
    
    K: Strike price
    r: Risk free rate
    T: Maturity date given as a fraction of a year
    S_t: A df containing stock price history
    """
    
    # Define the standard normal function
    N = norm.cdf
    
    # Define sigma
    sigma = stock_price.diff().std()[0]
    
    # Define d1 and d2
    d1 = (np.log(S_t.iloc[-1]/K) + (r+0.5*sigma**2) * (T)) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    # Calculate option price
    c_t = S_t.iloc[-1]*N(d1)-K*exp(-r*(T))*N(d2)
    
    # Return price
    return c_t[0]

In [6]:
## Call price for different values of K
call_price_list = []
for i in range(230):
    c = call_price(i, 0.1, 0.25, stock_price)
    call_price_list.append(c)

In [7]:
## Plot call price as function of K
call_price_df = pd.DataFrame(call_price_list, columns = ['Call Price'])
fig = px.line(call_price_df, title='Call price as a function of strike price (K)', 
              labels=dict(value="Call Price", index="$K$"))
fig.show()

In [8]:
## Define a option that will calculate price of eu put option on non dividend paying stock
def put_price(K, r, T, S_t):
    """
    This functions calculates the price on a european call option on a 
    non dividend paying stock and takes the arguments:
    
    K: Strike price
    r: Risk free rate
    T: Maturity date given as a fraction of a year
    S_t: A df containing stock price history
    """
    
    # Define the standard normal function
    N = norm.cdf
    
    # Define sigma
    sigma = stock_price.diff().std()[0]
    
    # Define d1 and d2
    d1 = (np.log(S_t.iloc[-1]/K) + (r+0.5*sigma**2) * (T)) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    # Calculate option price
    p_t = K*exp(-r*(T))*N(-d2)-S_t.iloc[-1]*N(-d1)
    
    # Return price
    return p_t[0]

In [9]:
## put price for different values of K
put_price_list = []
for i in range(230):
    p = put_price(i, 0.1, 0.25, stock_price)
    put_price_list.append(p)

In [10]:
## Plot put price as function of K
put_price_df = pd.DataFrame(put_price_list, columns = ['Put Price'])
fig = px.line(put_price_df, title='Put price as a function of strike price (K)', 
              labels=dict(value="Put Price", index="$K$"))
fig.show()

In [11]:
## Same graph
put_call_df = pd.concat([put_price_df,call_price_df])

In [14]:
fig = px.line(put_call_df, title='Put / Call price as a function of strike price (K)',
             labels=dict(value='Price', index='$K$'))
fig.show()